In [2]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [ ]:
base_path = "../../purple-hawk-data"
source_path = f"{base_path}/source"
dest_path = f"{base_path}/db-copy"

In [11]:
cd = gpd.read_file(f"{source_path}/nycd.csv")

In [17]:
cd_essentials = cd.reindex(["BoroCD", "the_geom"], axis=1)

In [21]:
cd_normal = cd_essentials
cd_normal["BoroCD"] = cd_essentials["BoroCD"].astype(str)
cd_normal["borough"] = cd_normal["BoroCD"].str[0]
cd_normal["code"] = cd_normal["BoroCD"].str[1:3]
cd_normal["the_geom"] = cd_normal.the_geom.apply(wkt.loads)

In [24]:
cd_spatial = gpd.GeoDataFrame(cd_normal, geometry="the_geom", crs=4326)

In [29]:
cd_wgs84 = cd_spatial.drop("BoroCD", axis=1)

In [30]:
cd_lift = cd_wgs84.to_crs(2263)

In [31]:
cd_published = pd.DataFrame(
    data={
        "borough": cd_normal["borough"],
        "code": cd_normal["code"],
        "wgs84": cd_wgs84["the_geom"],
        "lift": cd_lift["the_geom"],
    }
)

In [33]:
cd_published.to_csv(f"{dest_path}/cd_v1.csv", index=False)